# Detecting near duplicates using Jaccard similarity and MinHashing

In this notebook we explore how we can detect near duplicates in our data using the above two methods.

Note that a large part of this notebook is based of and recoded from this tutorial: https://towardsai.net/p/l/text-similarity-using-k-shingling-minhashing-and-lshlocality-sensitive-hashing

For those of you who will want to learn more about the subject and are familiar with `R` I highly recommend you also check the original tutorial.

In [2]:
import os
import re

# List and read content of all text files.
os.chdir("data")

files = []
for f in os.listdir("."):
    if f.endswith(".txt"):
        files.append(f)

documents = []
for file in files:
    with open(file, encoding="utf-8") as f:
        content = f.readlines()
        documents.append(content)

documents

[['The night is dark and the moon is red.\n'],
 ['I can see moon is red, the night is dark.\n'],
 ['The moon in the night is red.\n']]

In [3]:
def preprocess_text(doc):
    # Join lines, remove punctuation, convert to lowercase.
    text = re.sub(r"[^\w\s]", "", " ".join(doc)).lower()
    # Remove extra spaces.
    text = re.sub(r"\s+", " ", text).strip()
    # Split into words.
    words = text.split()
    return words

processed_documents = []
for doc in documents:
    processed_documents.append(preprocess_text(doc))

processed_documents[0]

['the', 'night', 'is', 'dark', 'and', 'the', 'moon', 'is', 'red']

In [6]:
# k-shingles of a document are all the different variations of words of length k found in it.
# The elements of the set of shingles are unique for each document.
def shingling(document, k):
    shingles = []
    for i in range(len(document) - k + 1):
        shingle = " ".join(document[i:i + k])
        shingles.append(shingle)
    return list(set(shingles)) # Return unique shingles.

k = 3
shingled_documents = []
for doc in processed_documents:
    shingled = shingling(doc, k)
    shingled_documents.append(shingled)

print(documents[0])
print(shingled_documents[0])

['The night is dark and the moon is red.\n']
['and the moon', 'dark and the', 'the moon is', 'the night is', 'is dark and', 'moon is red', 'night is dark']


Let's create a characteristic matrix that helps us understand the relationships between our three documents.
The rows of the matrix are the elements of the set of all unique shingles present across the entire content of the documents.
Each column of the matrix represents one of the documents.

In [9]:
import pandas as pd

# Flatten all shingles into a unique set (to get a universal set of shingles).
unique_shingles = set()
for doc in shingled_documents:
    unique_shingles.update(doc)
unique_shingles = sorted(unique_shingles)

# Characteristic matrix.
char_matrix = []

for doc in shingled_documents:
    row = []
    for shingle in unique_shingles:
        row.append(1 if shingle in doc else 0)
    char_matrix.append(row)

# Convert to df for easier visualization.
char_matrix_df = pd.DataFrame(char_matrix).T
char_matrix_df.columns = [f"doc_{i+1}" for i in range(len(shingled_documents))]
char_matrix_df.index = unique_shingles

char_matrix_df

,doc_1,doc_2,doc_3
and the moon,1,0,0
can see moon,0,1,0
dark and the,1,0,0
i can see,0,1,0
in the night,0,0,1
is dark and,1,0,0
is red the,0,1,0
moon in the,0,0,1
moon is red,1,1,0
night is dark,1,1,0


Now we need a way to measure how similar the documents are to each other.
We will do this using the Jaccard similarity measure, which is calculated as follows:

$$
J(A, B) = \frac{|A \cap B|}{|A \cup B|}
$$

We obtain the Jaccard similarity measure by dividing the intersection of the sets of two documents (the shingles that both documents contain) by the union of the two documents (the set of all unique shingles contained in both documents together).

In [10]:
import numpy as np

# Function to compute Jaccard similarity between two binary vectors.
def jaccard_similarity(x, y):
    intersection = np.sum(np.logical_and(x, y))  # Count common 1s
    union = np.sum(np.logical_or(x, y))  # Count total unique 1s
    return intersection / union if union != 0 else 0  # Avoid division by zero

# Compute Jaccard similarity distance matrix
def jaccard_similarity_matrix(char_matrix):
    n = char_matrix.shape[1]  # Number of documents
    similarity_matrix = np.zeros((n, n))

    for i in range(n):
        for j in range(n):
            similarity_matrix[i, j] = jaccard_similarity(char_matrix.iloc[:, i], char_matrix.iloc[:, j])

    return pd.DataFrame(similarity_matrix, index=char_matrix.columns, columns=char_matrix.columns)

In [11]:
# Compute Jaccard similarity netween two binary vectors.
def jaccard_similarity(x, y):
    intersection = np.sum(np.logical_and(x, y)) # Count common 1s.
    union = np.sum(np.logical_or(x, y)) # Count total unique 1s.
    return intersection / union if union != 0 else 0 # Avoid division by zero.

# Compute Jaccard similarity distance matrix.
def jaccard_similarity_matrix(char_matrix):
    n = char_matrix.shape[1] # We get nr. of documents.
    similarity_matrix = np.zeros((n, n))

    for i in range(n):
        for j in range(n):
            similarity_matrix[i, j] = jaccard_similarity(char_matrix.iloc[:, i], char_matrix.iloc[:, j]).round(2)

    return pd.DataFrame(similarity_matrix, index=char_matrix.columns, columns=char_matrix.columns)

jaccard_matrix = jaccard_similarity_matrix(char_matrix_df)

jaccard_matrix

,doc_1,doc_2,doc_3
doc_1,1.00,0.25,0.09
doc_2,0.25,1.00,0.08
doc_3,0.09,0.08,1.00


With a small amount of data, the above method works fine. The problem arises when dealing with a larger number of documents containing longer texts.

In this case, the characteristic matrix of unique shingles across all documents increases drastically, and calculating the Jaccard similarity measure on this matrix quickly becomes computationally impractical.

The name of the method that solves this problem is MinHashing.

MinHashing creates so-called signatures that provide more compact, computationally less demanding representations of the set of unique shingles. Using these signatures, we can also measure similarity between documents. Although this method yields a less precise estimate compared to the previously described method, its approximations are usually sufficient. The accuracy of the approximation increases with the number of signatures we choose.

If we want to create a MinHash of our characteristic matrix with 16 rows into 4 signatures, we must first generate 4 columns of randomly permuted rows that are independent from each other.

We do this with the hash function h(x) = (a * x + b) mod c.

Where the variables represent:

x => the row index (function argument).

a in b => random coefficients, less than or equal to x (the same coefficient must not repeat within the same signature but can repeat between signatures); they control the transformation of argument x.

c => a prime number (usually slightly larger than the number of rows; ensures the result stays within a certain range).

In [12]:
# Number of hash functions (signatures).
signature_num = 4

# Prime number slightly larger than the number of shingles.
prime = 17

# Total nr. of rows (shingles).
num_shingles = len(char_matrix_df)

np.random.seed(1)

# Generate unique coefficients a and b.
coeff_a = np.random.choice(range(1, num_shingles + 1), signature_num, replace=False)
coeff_b = np.random.choice(range(1, num_shingles + 1), signature_num, replace=False)

# Function to apply the hash function h(x) = (a * x + b ) mod c.
def generate_permutation(a, b, prime, num_shingles):
    hash_values = []
    for i in range(1, num_shingles + 1):
        hash_values.append((a * i + b) % prime)
    return hash_values

# Check the values of both coefficients.
print(coeff_a)
print(coeff_b)

[ 4 14  8  3]
[ 9  4 14 10]


We see that the coefficient values are appropriate (the repetition of number 14 is okay since it occurs between signatures and not within the same signature).

In [13]:
# Generate the permutation matrix.
permutations = {}
for i in range(signature_num):
    hash_col = generate_permutation(coeff_a[i], coeff_b[i], prime, num_shingles)
    permutations[f"hash_{i+1}"] = hash_col

# Convert to df for easier visualization.
permute_df = pd.DataFrame(permutations, index=range(1, num_shingles + 1))

permute_df

,hash_1,hash_2,hash_3,hash_4
1,13,1,5,13
2,0,15,13,16
3,4,12,4,2
4,8,9,12,5
5,12,6,3,8
6,16,3,11,11
7,3,0,2,14
8,7,14,10,0
9,11,11,1,3
10,15,8,9,6


Let's combine our original characteristic matrix with the hash values.

In [16]:
# Reset index of char_matrix_df.
char_matrix_df_reset = char_matrix_df.reset_index(drop=True)
# Set the index of char_matrix_df to match permute_df index (1 to 16).
char_matrix_df_reset.index = permute_df.index
# Concatenate both dataframes.
char_matrix_with_permutations = pd.concat([char_matrix_df_reset, permute_df], axis=1)

char_matrix_with_permutations

,doc_1,doc_2,doc_3,hash_1,hash_2,hash_3,hash_4
1,1,0,0,13,1,5,13
2,0,1,0,0,15,13,16
3,1,0,0,4,12,4,2
4,0,1,0,8,9,12,5
5,0,0,1,12,6,3,8
6,1,0,0,16,3,11,11
7,0,1,0,3,0,2,14
8,0,0,1,7,14,10,0
9,1,1,0,11,11,1,3
10,1,1,0,15,8,9,6


Now we can present the method of calculating the signature values. Let's start with the hash_1 column:

1) First, we look for the row within this column that contains the smallest number. We see that it is the second row, which contains the number 0.
2) On the left side of the same row, we check whether any of the documents have the number 1. We see that this is true for doc_2. Based on this finding, in our signature matrix (shown below), in the row minhash_1 and column doc_2, the value 0 will be entered.
3) Since the other two documents in the same row did not contain the number 1, we check which row within the hash_1 column contains the next smallest number. We find that it is row number 15. On the left side of this row, the number 1 is in the column doc_1, so in the minhash_1 row of the signature matrix, in the column doc_1, the value 3 will be entered. Since we have not yet found a match for doc_3 via number 1, we continue with the same process.
4) In row 11, the number 1 is in the doc_3 column, so in the minhash_1 row of the signature matrix, in the doc_3 column, the value 2 will be entered.

Now that we have located the number 1 for each of our documents, it means that via the first hash value we have generated signature values for all three of our documents.* These values are [1, 0, 2]. Using the same process, signature values will be generated for the other three hash_ columns as well. Let’s see how we can build the signature matrix that contains these values.

*In our example, the values of one nicely distributed across the documents at each step, but this is just the result of our specific case and not a rule. It can happen that for the smallest consecutive hash_ value, the number 1 appears under multiple documents simultaneously (see rows 9 and 10 or row 16). In such a case, the consecutive value from the hash_ column is copied into every doc_ column in the signature matrix that has not yet been filled in relation to the current hash_ value (see the minhash_ row below). For those already filled, the copying step is simply skipped.

In [17]:
# Build the signature matrix.
num_docs = char_matrix_df.shape[1]
signature_matrix = np.full((signature_num, num_docs), np.nan)

# Obtain the non-zero rows' indices for all columns.
non_zero_rows = []
for j in range(num_docs):
    non_zero_rows.append(np.where(char_matrix_df.iloc[:, j] != 0)[0])

# Compute the signature matrix.
for i in range(num_docs):
    for s in range(signature_num):
        signature_matrix[s, i] = np.min(permute_df.iloc[non_zero_rows[i], s])

# Convert to df.
signature_matrix_df = pd.DataFrame(signature_matrix,
                                   columns=[f"doc_{i+1}" for i in range(num_docs)],
                                   index=[f"minhash_{s+1}" for s in range(signature_num)]
                                   ).astype(int)

signature_matrix_df

,doc_1,doc_2,doc_3
minhash_1,1,0,2
minhash_2,1,0,5
minhash_3,1,1,0
minhash_4,2,3,0


In [18]:
# Compute signature similarity.
def sig_similarity(x, y):
    count = 0
    for i in range(len(x)):
        if x[i] == y[i]:
            count += 1
    return count / len(x)

# Compute pairwise similarity for the signature matrix.
def signature_similarity_matrix(signature_matrix_df):
    n = signature_matrix_df.shape[1]
    similarity_matrix = np.zeros((n, n))
    
    for i in range(n):
        for j in range(n):
            similarity_matrix[i, j] = sig_similarity(signature_matrix_df.iloc[:, i], signature_matrix_df.iloc[:, j])
    
    return pd.DataFrame(similarity_matrix, index=signature_matrix_df.columns, columns=signature_matrix_df.columns).round(2)

# Compute similarity matrix.
signature_similarity_df = signature_similarity_matrix(signature_matrix_df)

signature_similarity_df

/tmp/ipykernel_93607/3094329701.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if x[i] == y[i]:


,doc_1,doc_2,doc_3
doc_1,1.00,0.25,0.0
doc_2,0.25,1.00,0.0
doc_3,0.00,0.00,1.0


In [19]:
jaccard_matrix

,doc_1,doc_2,doc_3
doc_1,1.00,0.25,0.09
doc_2,0.25,1.00,0.08
doc_3,0.09,0.08,1.00


We see that the values of the Jaccard matrix and the similarity matrix of signature values do not match very precisely, which was to be expected with such a small number of hash functions or signatures. As we increase the number of these, the MinHash similarity values between documents stabilize and begin to closely approximate the Jaccard similarity measure with high accuracy. Let's try to interpret how the MinHash algorithm works to better understand why this similarity occurs.

With the MinHash algorithm, we simulate a random permutation of the union of all unique shingles across the documents (the permutation in MinHash is not truly random because the result is *deterministic* via the hash function). Above, in the hash_ columns within the char_matrix_with_permutations, we can see concrete examples of such permutations. We saw that the signature values are obtained by looking at in which document the shingle with the smallest value within each permutation appears. A match of shingles between two documents occurs in this scenario when both documents share that shingle. The probability of this match is equal to the probability that, in the union of shingles from both documents, when the shingles are randomly permuted and lined up, the first shingle comes from both documents. If we write this probability with a mathematical formula, we obtain an expression identical to the formula for the Jaccard similarity:

$$
P(MinHash(A) = MinHash(B)) = \frac{|A \cap B|}{|A \cup B|}
$$

If two documents share many shingles, the probability is higher that the first shingle "selected" by the hash function is one of the common shingles. As the number of MinHash signatures built from hash functions increases, the percentage of matching MinHash values between the documents approaches the Jaccard similarity measure.